### mnist 이미지데이터 (이미지 >> 매트릭스)

**다운로드**

In [1]:
import urllib.request as req
import gzip, os, os.path

savepath = "./mnist"
baseurl = "http://yann.lecun.com/exdb/mnist"
files = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"]

# 다운로드
if not os.path.exists(savepath): os.mkdir(savepath)
    
for f in files:
    url = baseurl + "/" + f
    loc = savepath + "/" + f
    print("download:", url)
    if not os.path.exists(loc):
        req.urlretrieve(url, loc)
        
# GZip 압축 해제
for f in files:
    gz_file = savepath + "/" + f
    raw_file = savepath + "/" + f.replace(".gz", "")
    print("gzip:", f)
    with gzip.open(gz_file, "rb") as fp:
        body = fp.read()
        with open(raw_file, "wb") as w:
            w.write(body)
print("ok")

download: http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
gzip: train-images-idx3-ubyte.gz
gzip: train-labels-idx1-ubyte.gz
gzip: t10k-images-idx3-ubyte.gz
gzip: t10k-labels-idx1-ubyte.gz
ok


**csv파일로 저장**

In [2]:
import struct
def to_csv(name, maxdata):
    # 레이블 파일과 이미지 파일 열기
    lbl_f = open("./mnist/"+name+"-labels-idx1-ubyte", "rb") #파일 읽음 (각각 빨대)
    img_f = open("./mnist/"+name+"-images-idx3-ubyte", "rb") #파일 읽음 (각각 빨대)
    csv_f = open("./mnist/"+name+".csv", "w", encoding="utf-8") #쓰기
    # 헤더 정보 읽기 --- (※1)
    mag, lbl_count = struct.unpack(">II", lbl_f.read(8)) # ">||" 그냥 기호, 옆으로 넘어가라는.. 문법임
    mag, img_count = struct.unpack(">II", img_f.read(8)) # 8bit로 받음 mag,lbl_count(img_count) 각각 4bit씩
    rows, cols = struct.unpack(">II", img_f.read(8))
    pixels = rows * cols
    # 이미지 데이터를 읽고 CSV로 저장하기 --- (※2)
    res = []
    for idx in range(lbl_count):       #lbl_count(1000개)만큼 읽어서
        if idx > maxdata: break       #idx는 카운트변수
        label = struct.unpack("B", lbl_f.read(1))[0]  
        bdata = img_f.read(pixels) #784개만큼 읽어오라,(실데이터 읽음)
        sdata = list(map(lambda n: str(n), bdata))
        csv_f.write(str(label)+",")
        csv_f.write(",".join(sdata)+"\r\n")
        # 잘 저장됐는지 이미지 파일로 저장해서 테스트하기 -- (※3)
        if idx < 10:
            s = "P2 28 28 255\n"
            s += " ".join(sdata)
            iname = "./mnist/{0}-{1}-{2}.pgm".format(name,idx,label)
            with open(iname, "w", encoding="utf-8") as f:
                f.write(s)
    csv_f.close()
    lbl_f.close()
    img_f.close()
# 결과를 파일로 출력하기 --- (※4)
to_csv("train", 1000) #숫자는 늘리면 됨
to_csv("t10k", 500)

**훈련코드**

In [13]:
from sklearn import svm, metrics #분류모형은 무조건 metrics.. 평가지표..

In [11]:
def load_csv(fname):
    labels=[] #정답지
    images=[] #속성들(features)
    
    with open(fname,"r") as f:
        for line in f: # \n기준으로 분리
            cols = line.split(",") #빈줄 없애는 작업
            if len(cols) <2: continue#빈줄 부분은 길이가 1이기 때문
            labels.append(int(cols.pop(0))) #cols.pop(0) label값 뽑는건데, 하나씩 뽑으면 문자로 인식함
                                            #따라서 int이용해 숫자로 바꿔줌
            vals = list(map(lambda n: int(n)/256, cols)) #정규화하는 작업.
                                                          #255대신 256을 쓴 이유는 정밀도 높이려고
            images.append(vals)
    return {"labels":labels, "images":images}


train = load_csv("./mnist/train.csv")
test = load_csv("./mnist/t10k.csv")

In [17]:
clf = svm.SVC()
clf.fit(train["images"],train["labels"])
#C값이 커질수록 이상치도 포함시키는 경향을 띔

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [23]:
pred = clf.predict(test["images"])

ac_score = metrics.accuracy_score(test["labels"],pred)
cl_report = metrics.classification_report(test["labels"],pred)
print("정답률 = ", ac_score)
print("리포트 = ")
print(cl_report)

# 정확도가 낮을때 아래 수치를 활용함
# ex) 아래 결과를 통해 3과 6이 5로 잘못 예측된다는 것을 유추할 수 있음
#precision(예측치 기준), recall(실측치 기준)

정답률 =  0.7884231536926147
리포트 = 
              precision    recall  f1-score   support

           0       0.87      0.93      0.90        42
           1       0.81      1.00      0.89        67
           2       0.84      0.69      0.76        55
           3       0.87      0.57      0.68        46
           4       0.76      0.75      0.75        55
           5       0.63      0.80      0.71        50
           6       0.97      0.67      0.79        43
           7       0.74      0.86      0.79        49
           8       0.91      0.72      0.81        40
           9       0.71      0.81      0.76        54

    accuracy                           0.79       501
   macro avg       0.81      0.78      0.78       501
weighted avg       0.80      0.79      0.79       501

